Environment Variables

In [2]:
import os
import getpass

os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)

faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
)

retriever = faiss_vectorstore.as_retriever()

In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [11]:
from langchain_google_genai import GoogleGenerativeAI

gemini_chat_model = GoogleGenerativeAI(model="gemini-1.5-pro")

In [12]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_generation_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | gemini_chat_model, "context": itemgetter("context")}
)

In [13]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "What is Retrieval Augmented Generation?"})

{'response': 'Retrieval-augmented generation (RAG) is a text generation paradigm that combines deep learning technology with traditional retrieval techniques. It leverages external information retrieval to provide additional context to large language models (LLMs) during the generation process. This approach helps overcome the limitations of LLMs that rely solely on their internal knowledge, leading to improved performance in various NLP tasks like:\n\n* **Dialogue response generation:** Generating more relevant and informative responses in conversations.\n* **Machine translation:** Enhancing translation quality by accessing relevant bilingual data.\n* **Abstractive summarization:** Creating concise summaries by incorporating information from multiple sources.\n* **Code generation:** Generating code by retrieving relevant code snippets and documentation. \n* **Paraphrase:**  Producing different versions of a text while preserving its meaning.\n* **Knowledge-intensive generation:** Gene

## LangGraph - Building Cyclic Applications with LangChain

> Add blockquote

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effetively allowing us to recreate appliation flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

In [14]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

In [15]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

/tmp/ipykernel_1187559/2535545004.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tool_belt)


In [17]:
from langchain_google_genai import GoogleGenerativeAI
model = GoogleGenerativeAI(model="gemini-1.5-pro",temperature=0)

In [18]:
from langchain_core.agents import create_gemini_functions_agent

functions = [create_gemini_functions_agent(t) for t in tool_belt]
model = model.bind_tools(functions)

AttributeError: 'GoogleGenerativeAI' object has no attribute 'bind_functions'